In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


from statsmodels.tsa.seasonal import seasonal_decompose

from neuralprophet import NeuralProphet

#To mute Warnings
import warnings
warnings.filterwarnings("ignore")

c:\Miniconda3\envs\intern\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [2]:
# Weekly
df_w_2020_23 = pd.read_csv('./data/VolumeDataset_2020_to_2023.csv')
#  Made Date index column 
dates = df_w_2020_23.Year*100+df_w_2020_23.Week
df_w_2020_23['Date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')
df_w_2020_23 = df_w_2020_23.set_index('Date')

# Neural Prpphet Prediction (Weekly)
    - N2 (High Varience/ High Mean)

## Total data 01-12-2020 to 24-07-2022 (135 rows)

q1
- train data : 2020-01-12 to 2021-12-26 (103 rows)
- test data : 2022-01-02 to 2022-03-20 (12 rows)

q2
- train data : 2020-01-12 to 2022-03-27 (116 rows)
- test data : 2022-04-03 to 2022-06-19 (12 rows)

q3
- train data : 2020-01-12 to 2022-05-01 (121 rows)
- test data : 2022-05-08 to 2022-07-24 (12 rows)

In [16]:
from datetime import datetime
temp_n2 = pd.read_excel('./data/temperature/temp_n2_w3_s2/N2_temp_2020_23.xlsx')
temp_n2['Date'] = pd.to_datetime(temp_n2[['Year', 'Month', 'Day']])

In [17]:
# Data
n2_data = df_w_2020_23.loc[:,['N2']].reset_index(drop=True)
n2_data['Date'] = df_w_2020_23.index
n2_data.columns = ['y','ds']
n2_data = n2_data[['ds', 'y']]
# n2_data.fillna(0.00,inplace=True)

n2_data = n2_data.drop(n2_data[n2_data.ds.duplicated(keep='first')].index, axis=0)
def fun_delhi(date):
    d = temp_n2[temp_n2.Date == date].delhi_avg.values
    return d[0] if d else 0
# n2_data['temp_delhi'] = n2_data.ds.apply(fun_delhi)

def fun_luknow(date):
    d = temp_n2[temp_n2.Date == date].laknow_avg.values
    return d[0] if d else 0

def fun_rajasthan(date):
    d = temp_n2[temp_n2.Date == date].rajasthan_avg.values
    return d[0] if d else 0
n2_data['temp_rajasthan'] = n2_data.ds.apply(fun_rajasthan)

In [18]:
confidence_level = 0.9

boundaries = round((1 - confidence_level) / 2, 2)
# NeuralProphet only accepts quantiles value in between 0 and 1
quantiles = [boundaries, confidence_level + boundaries]

In [46]:
q1 = n2_data[:115] #
q1_n2_ja = NeuralProphet(yearly_seasonality=True, weekly_seasonality=False, n_changepoints=4, n_lags=4,
    n_forecasts = 12,
    learning_rate=0.01, quantiles=quantiles)

q1_n2_ja.add_future_regressor("temp_rajasthan")
q1_n2_ja.add_country_holidays(country_name="IN")
q1_metrics =  q1_n2_ja.fit(q1[:103],freq='W', epochs=850)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.029% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 8


Epoch 850: 100%|██████████| 850/850 [00:00<00:00, 11903.98it/s, loss=0.0227, v_num=184, MAE=3.77e+4, RMSE=4.97e+4, Loss=0.0228, RegLoss=0.000]  


In [47]:
q1_future = q1_n2_ja.make_future_dataframe(q1[:103], periods=12, n_historic_predictions=True, regressors_df=q1.drop('y', axis=1)[103:115])
q1_forecast = q1_n2_ja.predict(q1_future)
q1_forecast['y']=pd.concat([q1['y']], axis=0, ignore_index=True)

INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.029% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.13% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.13% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.data.processing._handle_missing_data) - Dropped 12 rows at the end with NaNs in 'y' column.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 99.38it/s] 


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [48]:
q1_n2_ja.highlight_nth_step_ahead_of_each_forecast(12)
q1_n2_ja.plot(q1_forecast)

ERROR - (NP.plotly.plot) - plotly-resampler is not installed. Please install it to use the resampler.


In [49]:
q1_forecast.iloc[100:, [0,1,10,11,12,13]]

,ds,y,yhat9,yhat10,yhat11,yhat12
100,2021-12-12,145646.87,98134.132812,119442.726562,114136.859375,143701.75000
101,2021-12-19,144741.66,125653.265625,136963.781250,148183.312500,147220.03125
102,2021-12-26,187316.23,160328.312500,125332.796875,131086.000000,153983.93750
103,2022-01-02,150460.35,184242.281250,159349.812500,125107.367188,134026.68750
104,2022-01-09,100759.70,186678.437500,187855.531250,167543.000000,137181.68750
105,2022-01-16,213987.46,206477.781250,204811.312500,207025.875000,187726.25000
106,2022-01-23,236690.90,235206.687500,221628.687500,222375.125000,224142.00000
107,2022-01-30,244258.45,254594.531250,246541.968750,237659.562500,236193.00000
108,2022-02-06,330786.82,288322.187500,279091.218750,275693.562500,265768.96875
109,2022-02-13,255086.80,288812.250000,286076.187500,282208.375000,277245.59375


In [8]:
q2 = n2_data[:128] #
q2_n2_ja = NeuralProphet(yearly_seasonality=True, weekly_seasonality=False, n_changepoints=4, n_lags=4,
    n_forecasts = 12,
    learning_rate=0.01, quantiles=quantiles)

q2_n2_ja.add_future_regressor("temp_rajasthan")
q2_n2_ja.add_country_holidays(country_name="IN")
q2_metrics =  q2_n2_ja.fit(q2[:116],freq='W', epochs=850)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.138% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16


Epoch 850: 100%|██████████| 850/850 [00:00<00:00, 10882.59it/s, loss=0.0208, v_num=178, MAE=3.6e+4, RMSE=4.74e+4, Loss=0.0199, RegLoss=0.000]   


In [10]:
q2_future = q2_n2_ja.make_future_dataframe(q2[:116], periods=12, n_historic_predictions=True, regressors_df=q2.drop('y', axis=1)[116:128])
q2_forecast = q2_n2_ja.predict(q2_future)
q2_forecast['y']=pd.concat([q2['y']], axis=0, ignore_index=True)

INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.138% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.219% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.219% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.data.processing._handle_missing_data) - Dropped 12 rows at the end with NaNs in 'y' column.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 93.49it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [27]:
q2_forecast.iloc[116:, [0,1,10,11,12,13]]

,ds,y,yhat9,yhat10,yhat11,yhat12
116,2022-04-03,316574.82,459467.93750,460137.06250,458607.59375,470408.96875
117,2022-04-10,389310.45,438090.15625,466171.50000,460809.40625,461623.43750
118,2022-04-17,316291.99,486975.96875,466651.50000,494891.68750,490316.37500
119,2022-04-24,174196.16,410088.15625,405673.15625,393084.43750,414119.28125
120,2022-05-01,50814.07,357779.21875,378520.59375,368057.84375,359289.59375
121,2022-05-08,88372.79,274588.84375,279993.15625,298841.84375,289660.28125
122,2022-05-15,66021.34,176714.46875,222744.53125,219492.34375,234879.59375
123,2022-05-22,42428.66,194440.37500,156143.03125,185141.18750,185395.81250
124,2022-05-29,62878.58,238253.37500,181074.53125,166351.56250,189745.71875
125,2022-06-05,61189.40,NaN,269353.37500,219103.09375,207770.25000


In [15]:
q2_n2_ja.highlight_nth_step_ahead_of_each_forecast(12)
q2_n2_ja.plot(q2_forecast)

ERROR - (NP.plotly.plot) - plotly-resampler is not installed. Please install it to use the resampler.


In [50]:
q3 = n2_data[:133] #
q3_n2_ja = NeuralProphet(yearly_seasonality=True, weekly_seasonality=False, n_changepoints=4, n_lags=4,
    n_forecasts = 12,
    learning_rate=0.01, quantiles=quantiles)

q3_n2_ja.add_future_regressor("temp_rajasthan")
q3_n2_ja.add_country_holidays(country_name="IN")
q3_metrics =  q3_n2_ja.fit(q3[:121],freq='W', epochs=850)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.174% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16


Epoch 850: 100%|██████████| 850/850 [00:00<00:00, 16875.21it/s, loss=0.0284, v_num=185, MAE=4.26e+4, RMSE=5.61e+4, Loss=0.0286, RegLoss=0.000]  


In [51]:
q3[125:133]

,ds,y,temp_rajasthan
127,2022-06-05,61189.40,24.2
128,2022-06-12,47457.48,23.6
129,2022-06-19,92917.26,23.8
130,2022-06-26,197860.26,22.4
131,2022-07-03,120700.73,24.6
132,2022-07-10,63340.35,21.4
133,2022-07-17,72871.29,22.3
134,2022-07-24,171832.27,23.2


In [52]:
q3_future = q3_n2_ja.make_future_dataframe(q3[:121], periods=12, n_historic_predictions=True, regressors_df=q3.drop('y', axis=1)[121:133])
q3_forecast = q3_n2_ja.predict(q3_future)
q3_forecast['y']=pd.concat([q3['y']], axis=0, ignore_index=True)

INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.174% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.248% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.248% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.data.processing._handle_missing_data) - Dropped 12 rows at the end with NaNs in 'y' column.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 70.72it/s]


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [53]:
q3_n2_ja.highlight_nth_step_ahead_of_each_forecast(12)
q3_n2_ja.plot(q3_forecast)

ERROR - (NP.plotly.plot) - plotly-resampler is not installed. Please install it to use the resampler.


In [56]:
q3_forecast.iloc[116:, [0,1,10,11,12,13]]

,ds,y,yhat9,yhat10,yhat11,yhat12
116,2022-04-03,316574.82,379469.343750,393250.343750,390124.750000,389872.062500
117,2022-04-10,389310.45,398682.843750,404448.437500,412567.843750,412126.750000
118,2022-04-17,316291.99,425034.375000,413334.437500,418941.500000,431073.500000
119,2022-04-24,174196.16,351665.937500,359457.812500,344019.031250,342169.187500
120,2022-05-01,50814.07,41141.859375,51401.828125,52980.828125,48783.773438
121,2022-05-08,88372.79,222969.875000,238213.843750,245788.343750,250635.406250
122,2022-05-15,66021.34,136914.484375,177693.343750,182596.625000,181082.031250
123,2022-05-22,42428.66,133484.578125,101978.476562,127070.625000,137633.062500
124,2022-05-29,62878.58,173629.062500,163022.656250,128909.953125,149948.562500
125,2022-06-05,61189.40,229339.093750,217183.187500,197970.968750,184029.750000


In [75]:
q0 = n2_data[:103] #
q0['temp_delhi'] = q0.ds.apply(fun_delhi)
q0_n2_ja = NeuralProphet(yearly_seasonality=True, weekly_seasonality=False, n_changepoints=4, n_lags=4,
    n_forecasts = 12,
    learning_rate=0.01, quantiles=quantiles)

q0_n2_ja.add_future_regressor("temp_rajasthan")
q0_n2_ja.add_future_regressor("temp_delhi")
# q0_n2_ja.add_future_regressor("temp_rajasthan")
q0_n2_ja.add_country_holidays(country_name="IN")
q0_metrics =  q0_n2_ja.fit(q0[:91],freq='W', epochs=850)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 98.901% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 8


Epoch 850: 100%|██████████| 850/850 [00:00<00:00, 11006.09it/s, loss=0.0138, v_num=187, MAE=2.98e+4, RMSE=3.97e+4, Loss=0.0137, RegLoss=0.000]  


In [76]:
q0_future = q0_n2_ja.make_future_dataframe(q0[:91], periods=12, n_historic_predictions=True, regressors_df=q0.drop('y', axis=1)[91:103])
q0_forecast = q0_n2_ja.predict(q0_future)
q0_forecast['y']=pd.concat([q0['y']], axis=0, ignore_index=True)

INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 98.901% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.029% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.029% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO - (NP.data.processing._handle_missing_data) - Dropped 12 rows at the end with NaNs in 'y' column.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 33.02it/s]


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [72]:
q0_forecast.iloc[91:, [0,1,10,11,12,13]]

,ds,y,yhat9,yhat10,yhat11,yhat12
91,2021-10-10,121396.89,188424.21875,192946.43750,189537.71875,184114.93750
92,2021-10-17,124869.03,198388.46875,202002.46875,201051.12500,195874.84375
93,2021-10-24,119091.80,189723.96875,202822.43750,204729.15625,207166.62500
94,2021-10-31,54177.17,176038.09375,202717.93750,211628.62500,214155.00000
95,2021-11-07,55924.59,205585.65625,195163.12500,210265.06250,220098.43750
96,2021-11-14,60717.78,222970.12500,207280.96875,202797.09375,222836.62500
97,2021-11-21,51985.34,209324.34375,242782.03125,238003.31250,230573.75000
98,2021-11-28,60546.02,170090.28125,240309.78125,251259.56250,240508.18750
99,2021-12-05,110581.57,212305.15625,220430.34375,238490.43750,262158.06250
100,2021-12-12,145646.87,NaN,226318.31250,233028.37500,279944.12500


In [73]:
q0_n2_ja.highlight_nth_step_ahead_of_each_forecast(12)
q0_n2_ja.plot(q0_forecast)

ERROR - (NP.plotly.plot) - plotly-resampler is not installed. Please install it to use the resampler.
